In [16]:
import pandas as pd
from itertools import combinations as comb
import csv

def fetch_synergies_from_file(drug_ids, csv_file_path="drug_data.csv"):
    """Fetch and return synergy scores for analysis from a CSV file."""
    synergy_results = {}
    drug_data = pd.read_csv(csv_file_path)

    pairs = comb(drug_ids, 2)  # Generate all unique pairs from the given drug IDs

    for id1, id2 in pairs:
        # Find the combination in the dataframe
        row = drug_data[(drug_data['idDrugA'].isin([id1, id2])) & (drug_data['idDrugB'].isin([id1, id2]))]
        if not row.empty:
            scores = {
                'bliss': row.iloc[0]['bliss'],
                'zip': row.iloc[0]['zip'],
                'loewe': row.iloc[0]['loewe'],
                'hsa': row.iloc[0]['hsa']
            }
        else:
            scores = "Combination Data Not Found"
        
        synergy_results[(id1, id2)] = scores

    return synergy_results

import numpy as np

def analyze_combination_synergy(synergy_data):
    """Analyze synergy focusing on Loewe scores, with immediate checks for ZIP and HSA."""
    overall_synergy_responses = {}
    any_antagonism = False
    any_significant_synergy = False

    for pair, scores in synergy_data.items():
        loewe = scores['loewe']
        zip_score = scores['zip']
        hsa = scores['hsa']
        bliss = scores['bliss']

        # Check for very high HSA indicating strong synergy
        if hsa > 20:  # Example threshold for HSA indicating strong synergy
            any_significant_synergy = True

        # Evaluate Loewe score for synergy or antagonism
        if loewe > 0:
            overall_synergy_responses[pair] = "Antagonistic"
        elif loewe < 0:
            overall_synergy_responses[pair] = "Synergistic"

    overall_result = "Synergistic" if (all(response == "Synergistic" for response in overall_synergy_responses.values()) or any_significant_synergy) else "Antagonistic"

    return overall_result


def save_overall_responses_to_csv(overall_responses, output_csv_path):
    """Save the overall synergy responses of drug combinations to a CSV file."""
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Drug Combination", "Overall Synergy Response"])
        for combination, response in overall_responses.items():
            writer.writerow([', '.join(map(str, combination)), response])

def read_drug_combinations(input_file_path):
    """Read drug combinations from a CSV file."""
    combinations = []
    with open(input_file_path, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Convert the string of drug IDs into a list of integers
            drug_ids = [int(id.strip()) for id in row['Drug Combination'].split(',')]
            combinations.append(drug_ids)
    return combinations


In [17]:
# Process
input_file_path = "valid_drug_combinations.csv"  # Adjust this to your actual CSV file path
output_csv_path = "synergy_test_results.csv"

combinations = read_drug_combinations(input_file_path)
print(combinations)
overall_responses = {}

[[168, 131, 175, 182, 87], [184, 140, 168, 168], [110, 165, 177], [168, 173, 183, 76, 68], [171, 168], [131, 131], [106, 178, 24, 182, 182], [132, 45, 177], [24, 163, 130, 171], [167, 169, 131], [77, 173, 182, 45, 68], [140, 68, 175, 179], [41, 132, 177], [177, 169], [24, 165, 132, 132], [168, 116], [162, 87], [163, 177], [169, 167, 162, 182, 179], [179, 106, 166, 130, 174], [140, 41, 132, 168, 172], [164, 2, 77, 173], [162, 164, 68], [131, 119, 169, 24, 170], [184, 165, 171], [181, 180, 41], [181, 110, 140], [180, 165], [106, 76, 68, 116], [34, 174], [172, 175, 106], [182, 2, 180], [2, 132, 162], [168, 174, 182, 173, 24], [172, 133], [164, 68, 110], [178, 24, 182, 175], [164, 177], [140, 177, 183, 173], [132, 174], [24, 2, 181, 170], [116, 176, 76, 162, 184], [133, 87, 45, 169], [34, 87, 176, 174], [68, 170, 167], [140, 181], [162, 168, 165], [131, 176, 116, 176, 162], [167, 176, 106, 181, 130], [177, 131], [166, 179, 169, 133], [165, 110, 110, 87], [170, 116, 178], [110, 164, 182, 34

In [18]:
for combination in combinations:
    pairwise_synergies = fetch_synergies_from_file(combination)
    overall_response = analyze_combination_synergy(pairwise_synergies)
    overall_responses[tuple(combination)] = overall_response

save_overall_responses_to_csv(overall_responses, output_csv_path)

#Optionally, load and print the DataFrame for review
df = pd.read_csv(output_csv_path)
print(df.head())

         Drug Combination Overall Synergy Response
0  168, 131, 175, 182, 87              Synergistic
1      184, 140, 168, 168              Synergistic
2           110, 165, 177              Synergistic
3   168, 173, 183, 76, 68              Synergistic
4                171, 168              Synergistic
